In [412]:
from PyPDF2 import PdfFileReader, PdfFileWriter
import os
import pathlib
import pandas as pd
import numpy as np
#import xlsxwriter
#from office365.runtime.auth.authentication_context import AuthenticationContext
#from office365.sharepoint.client_context import ClientContext
#from office365.sharepoint.files.file import File

In [413]:
# initialerstellung der csv mit allen original PDF's
# ToDo: Veröffentlichungsdatum hinzufügen
def read_pdf_meta(path):
    pdf_names = []
    meta_author = []
    meta_title = []
    meta_date = []
    meta_check = []
    meta_list = []
    files_path = list(path.glob('*.pdf'))
    is_checked = True
    meta_control = pd.read_excel(str(path) + '/metadata_list.xlsx') # was wenn Datei noc nicht existiert?
    
    for file in files_path:
        cur_reader = PdfFileReader(file)
        cur_pdf_name = pathlib.Path(file).name
        
        if(cur_pdf_name in meta_control.values):
            cur_index = meta_control.loc[meta_control['PDF'] == cur_pdf_name]
            if(pd.isna(cur_index.Check.iloc[0])):
                is_checked = False
                
        if(cur_pdf_name not in meta_control.values):
            pdf_names.append(cur_pdf_name)
            meta_title.append(cur_reader.getDocumentInfo()['/Title'])
            meta_author.append(cur_reader.getDocumentInfo()['/Author'])
            meta_date.append(cur_reader.getDocumentInfo()['/CreationDate'])
            meta_check.append('')

# Erstellung .csv mit Papern (Titel, Autoren)
            meta_list = {'PDF': pdf_names,
                        'Titles': meta_title,
                        'Authors': meta_author,
                        'Date': meta_date,
                        'Check': meta_check}
            
            df_meta = pd.DataFrame(meta_list)
            meta_control = pd.concat([meta_control, df_meta])
            meta_control.to_excel(str(path) + '/metadata_list.xlsx', index=False, header=True)
            os.system("start EXCEL.EXE " + str(path) + "/metadata_list.xlsx")
            raise Exception('Anpassen der Metadaten wegen neuer Datei notwendig. Datei wird geöffnet.\n\t\tAnschließend Programm neustarten.')
        
    if(not is_checked):
        os.system("start EXCEL.EXE " + str(path) + "/metadata_list.xlsx")
        raise Exception('Anpassen der Metadaten in Datei notwendig. Datei wird geöffnet.\n\t\tAnschließend Programm neustarten.')

    return(meta_control)

In [414]:
# Anpassung der Metadaten:
def write_pdf_meta(path):
    
#Einlesen der Excel
    excel_meta = pd.read_excel(str(path) + '/metadata_list.xlsx')

    for x in range(len(excel_meta)):
        cur_pdf = excel_meta['PDF'][x]
        cur_pdf_full = pdf_path + cur_pdf
        cur_reader = PdfFileReader(cur_pdf_full)
        cur_writer = PdfFileWriter()
        cur_writer.appendPagesFromReader(cur_reader)
        cur_metadata = cur_reader.getDocumentInfo()
        cur_writer.addMetadata(cur_metadata)
        cur_writer.addMetadata({"/Title": excel_meta['Titles'][x]})
        cur_writer.addMetadata({"/Author": excel_meta['Authors'][x]})
        cur_writer.addMetadata({"/CreationDate": str(excel_meta['Date'][x])})
        with open(cur_pdf_full, "wb") as fp:
            cur_writer.write(fp)


In [415]:
pdf_path = 'C:/Users/Jana/OneDrive/Dokumente/Desktop/Uni/Master/Thesis/PaperDatenextraktion/'
p = pathlib.Path(pdf_path)
df_pdf_meta = read_pdf_meta(p)
write_pdf_meta(pdf_path)

In [416]:
pdf = 'C:/Users/Jana/OneDrive/Dokumente/Desktop/Uni/Master/Thesis/PaperDatenextraktion/IARJSET7.pdf'
reader = PdfFileReader(pdf)
test = reader.getDocumentInfo()
print(test)

{'/Producer': 'Microsoft® Office Word 2007', '/Title': 'Introduction to IOT', '/Author': 'Pradyumna Gokhale, Omkar Bhat, Sagar Bhat', '/Subject': 'Research Methodology', '/Keywords': 'STP', '/Creator': 'Microsoft® Office Word 2007', '/CreationDate': '2018', '/ModDate': 'D:20180207133638', '/rgid': 'PB:330114646_AS:711017873043458@1546531279502', '/title': 'Introduction to IOT'}
